In [ ]:
import tensorflow as tf
import numpy as np
import nltk
import math


In [ ]:
print(tf.__version__)

In [2]:
import time
import util

batch_size=8



queries=util.load_our_queries_morp()
print(len(queries))

293


In [3]:
print(queries[3])

[['천왕/NNG', '성/NNG', '의/JKG', '마지막/NNG', '주요/NNG', '위성/NNG', '미란다/NNP', '를/JKO', '발견/NNG', '하/XSV', 'ㄴ/ETM', '연도/NNG', '는/JX', '?/SF'], ['1948/SN'], 5]


In [4]:
train=[]
for idx in range(len(queries)):
    train.append([util.load_stories_with_our_query(queries[idx]),queries[idx][0],queries[idx][1],1,queries[idx][2]])
    

In [5]:
print(train[0])

[[['팬젠더/NNP', '.'], ['팬젠더/NNP', '(/SS', 'pangender/SL', ')/SS', '는/JX', '자신/NNG', '이/JKS', '모든/MM', '젠더/NNG', '에/JKB', '속/NNG', '하/XSV', 'ㄴ다고/EC', '자각/NNG', '하/XSV', '는/ETM', '정체/NNG', '성/XSN', '이/VCP', '다/EF', './SF', '.'], ['자신/NNG', '이/JKS', '여러/MM', '가지/NNB', '젠더/NNG', '를/JKO', '오가/VV', 'ㄴ다고/EC', '자각/NNG', '하/XSV', '는/ETM', '젠더플루이드/NNP', '와/JKB', '는/JX', '완전히/MAG', '일치/NNG', '하/XSV', '지/EC', '않/VX', '으며/EC', '어느/MM', '정도/NNG', '구분/NNG', '되/XSV', 'ㄴ다/EF', './SF', '.'], ['</SS', 'br/SL', '//SP', '>/SS', '분류/NNG', ':/SP', '사회/NNG', '적/XSN', '성/NNG', ',/SP', '젠더퀴/NNP', '어/EC', '.']], ['자신/NNG', '이/JKS', '모든/MM', '젠더/NNG', '에/JKB', '속/NNG', '하/XSV', 'ㄴ다고/EC', '자각/NNG', '하/XSV', '는/ETM', '정체/NNG', '성/XSN', '은/JX', '무엇/NP', '이/VCP', 'ㄴ가/EF', '?/SF'], ['팬젠더/NNP'], 1, 1]


In [6]:
vocab=[]
story_max=0

for stories,query,answer,flag,idx in train:
    story_max=max(story_max,len(stories))
    for sent in stories:
        for word in sent:
            vocab.append(word)
    for word in query:
        vocab.append(word)
    for word in answer:
        vocab.append(word)
vocab=list(set(vocab))

print(len(vocab))
print(story_max)

14071
169


In [7]:
gpu_options = tf.GPUOptions(allow_growth = True)
with tf.Graph().as_default(),tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options,allow_soft_placement = True)) as sess:
    print("Build model ...")
    a=MN(vocab,64,300,story_max,3,batch_size,sess)
    print("Start training ...")
    a.batch_fit(train)

Build model ...
Start training ...
epoch: 10
total loss: 1483.50985527
232
87
0.375
#############################
56
0
0.0
#############################
#############################
epoch: 20
total loss: 938.493318558
232
117
0.5043103448275862
#############################
56
0
0.0
#############################
#############################
epoch: 30
total loss: 1436.18235064
232
116
0.5
#############################
56
0
0.0
#############################
#############################
epoch: 40
total loss: 48.9592175999
232
230
0.9913793103448276
#############################
56
0
0.0
#############################
#############################
epoch: 50
total loss: 0.220176851493
232
232
1.0
#############################
56
0
0.0
#############################
#############################
epoch: 60
total loss: 0.106906928762
232
232
1.0
#############################
56
0
0.0
#############################
#############################
epoch: 70
total loss: 1531.77592107
232
200
0.8620

In [1]:
import tensorflow as tf
import numpy as np
import nltk
import math
import time
import util
import pickle


class MN():
    def __init__(self,vocab,memory_dim,memory_max,story_max,hop,batch_size,session):
        self.vocab=vocab
        
        self.story_max=story_max
        
        self.hop=hop
        self.batch_size=batch_size
        
        self.memory_max=memory_max
        self.memory_dim=memory_dim
        #self.init_memory()
        self.memory=self.init_memory()
        
        #self.initializer=tf.random_normal_initializer(stddev=0.1)
        self.session=session
        self.init_tf_variable()
        
        init_tf=tf.initialize_all_variables()
        
        self.session.run(init_tf)
        
        
        
    def return_memory(self):
        return self.memory
    
    def init_tf_variable(self):
        #self.story_embedding_W=tf.Variable(self.initializer)
        self.story_embedding_W=tf.Variable(tf.truncated_normal([len(self.vocab),self.memory_dim],stddev=1.0/math.sqrt(float(len(self.vocab)))),name="story_embedding_W")
        #self.story_embedding_W=tf.Variable(tf.truncated_normal([len(self.vocab),self.memory_dim],stddev=0.1),name="story_embedding_W")
        #self.story_embedding_b=tf.Variable(tf.zeros([self.memory_dim]),name="story_embedding_b")

        self.query_embedding_W=tf.Variable(tf.truncated_normal([len(self.vocab),self.memory_dim],stddev=1.0/math.sqrt(float(len(self.vocab)))),name="query_embedding_W")
        #self.query_embedding_W=tf.Variable(tf.truncated_normal([len(self.vocab),self.memory_dim],stddev=0.1),name="query_embedding_W")
        #self.query_embedding_b=tf.Variable(tf.zeros([self.memory_dim]),name="query_embedding_b")

        self.output_W=tf.Variable(tf.truncated_normal([self.memory_dim,len(self.vocab)],stddev=1.0/math.sqrt(float(self.memory_dim))),name="output_W")
        #self.output_W=tf.Variable(tf.truncated_normal([self.memory_dim,len(self.vocab)],stddev=0.1),name="output_W")
        #self.output_b=tf.Variable(tf.zeros([len(self.vocab)]),name="output_b")

        self.TA = tf.Variable(tf.truncated_normal([self.story_max,self.memory_dim],stddev=0.1))

        self.story_batch=tf.placeholder(tf.float32,[self.batch_size,self.story_max,len(self.vocab)])
        self.query_batch=tf.placeholder(tf.float32,[self.batch_size,1,len(self.vocab)])
        self.answer_batch=tf.placeholder(tf.float32,[self.batch_size,1,len(self.vocab)])
        self.memory_init_flag_batch=tf.placeholder(tf.int32,[self.batch_size])


        self.cross_entropy_sum=tf.constant(0.0)
        self.predict_list=tf.constant([[0.0]*len(self.vocab)])
        self.cross_entropy_list=tf.constant([])
        linear_start_cross_entropy_list=tf.constant([])
        self.attention_list=[]
        for idx in range(self.batch_size):
            temp_attention_list=[]
            #print(tf.constant(1))
            #print()

            #self.memory=tf.mul(self.memory,tf.sub(tf.constant(1.0),tf.cast(self.memory_init_flag_batch[idx],dtype=tf.float32)))
            #self.memory_pointer=tf.mul(self.memory_pointer,tf.sub(tf.constant(1),self.memory_init_flag_batch[idx]))
            #self.embedding_story=tf.matmul(self.story_batch[idx],self.story_embedding_W)+self.story_embedding_b
            #embedding_query=tf.matmul(self.query_batch[idx],self.query_embedding_W)+self.query_embedding_b
            #self.memory=tf.cond(tf.equal(self.memory_init_flag_batch[idx],tf.constant(1)),self.init_memory,self.return_memory)
            self.embedding_story=tf.add(tf.matmul(self.story_batch[idx],self.story_embedding_W),self.TA) 
            embedding_query=tf.matmul(self.query_batch[idx],self.query_embedding_W)
            #tf.scatter_update(self.memory,self.memory_pointer,self.embedding_story)

            #self.memory=tf.concat(0,[self.memory,self.embedding_story])
            """
            for elem in tf.split(0,self.embedding_story.get_shape()[0],self.embedding_story):
                #print(self.memory)
                self.memory=tf.concat(0,[self.memory,elem])
                #self.memorytf.reshape(elem,[-1])
                #self.memory_pointer=tf.add(self.memory_pointer,1)
            """
            #self.memory=tf.concat(self.memory,self.embedding_story)
            self.u=embedding_query
            linear_start_u=self.u
            for i in range(self.hop):

                self.attention=tf.nn.softmax(tf.reshape(tf.reduce_sum(tf.mul(self.embedding_story,self.u),1),[1,-1]))
                temp_attention_list.append(self.attention[0])
                self.attended_embedding=tf.matmul(self.attention,self.embedding_story)
                self.u=tf.add(self.u,self.attended_embedding)

                linear_start_attention=tf.reshape(tf.reduce_sum(tf.mul(self.embedding_story,self.u),1),[1,-1])
                linear_start_attended_embedding=tf.matmul(linear_start_attention,self.embedding_story)
                linear_start_u=tf.add(linear_start_u,linear_start_attended_embedding)
                

            predict=tf.matmul(self.u,self.output_W)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(predict, tf.cast(self.answer_batch[idx], tf.float32), name="cross_entropy")
            self.cross_entropy_list=tf.concat(0,[self.cross_entropy_list,cross_entropy])
            
            linear_start_predict=tf.matmul(linear_start_u,self.output_W)
            linear_start_cross_entropy=tf.nn.softmax_cross_entropy_with_logits(linear_start_predict, tf.cast(self.answer_batch[idx], tf.float32), name="cross_entropy")
            linear_start_cross_entropy_list=tf.concat(0,[linear_start_cross_entropy_list,linear_start_cross_entropy])
            
            self.predict_list=tf.concat(0,[self.predict_list,tf.nn.softmax(predict)])
            self.attention_list.append(temp_attention_list)
        self.predict_list=self.predict_list[1:]
        self.cross_entropy_sum=tf.reduce_sum(self.cross_entropy_list)
        self.linear_start_cross_entropy_sum=tf.reduce_sum(linear_start_cross_entropy_list)

        story_embedding_W_hist=tf.histogram_summary("story_embedding_W",self.story_embedding_W)
        #story_embedding_b_hist=tf.histogram_summary("story_embedding_b",self.story_embedding_b)
        query_embedding_W_hist=tf.histogram_summary("query_embedding_W",self.query_embedding_W)
        #query_embedding_b_hist=tf.histogram_summary("query_embedding_b",self.query_embedding_b)
        output_W_hist=tf.histogram_summary("output_W",self.output_W)
        #output_b_hist=tf.histogram_summary("output_b",self.output_b)
        cross_entropy_sum_scalar = tf.scalar_summary("cross_entropy_sum",self.cross_entropy_sum)
        self.merged=tf.merge_all_summaries()
        self.writer=tf.train.SummaryWriter("/home/asd36952/tfbd/memnet/Memorynet_our_data_hop_batch_linear_start",self.session.graph)

        self.train_step=tf.train.AdamOptimizer(learning_rate=1e-2).minimize(self.cross_entropy_sum)
        self.linear_start_train_step=tf.train.AdamOptimizer(learning_rate=1e-2).minimize(self.linear_start_cross_entropy_sum)
                #self.train_step=tf.contrib.layers.optimize_loss(loss=self.cross_entropy_sum,learning_rate=1e-2,optimizer='Adam',gradient_noise_scale=1e-4,global_step=tf.contrib.framework.get_global_step())
                #grads_and_vars = tf.train.AdamOptimizer(learning_rate=1e-2).compute_gradients(self.cross_entropy_sum)
                #grads_and_vars = [(tf.clip_by_norm(g, 40), v) for g,v in grads_and_vars]
                #self.train_step = tf.train.AdamOptimizer(learning_rate=1e-2).apply_gradients(grads_and_vars)
    
    def batch_fit(self,train,epoch=100):
        train_dict=dict()
        for elem in train:
            train_dict[elem[-1]]=elem[:-1]
        train_data,valid_data=self.preprocessing(train)
        num_train=np.array(train_data).shape[0]
        num_valid=np.array(valid_data).shape[0]
        
        linear_start_flag=True
        before_loss=np.inf
        for ep in range(epoch):
            np.random.shuffle(train_data)
            for idx in range(int(num_train/self.batch_size)):
                current_time=time.time()
                if num_train-idx*self.batch_size<self.batch_size:
                    break
                batch_train=train_data[idx*self.batch_size:(idx+1)*self.batch_size]
                #print(np.array(batch_train)[:,0].tolist())
                #print(np.array(np.array(np.array(batch_train)[:,1].tolist()).reshape(-1,1,len(self.vocab)).tolist()).shape)
                if linear_start_flag==True:
                    summary,_=self.session.run([self.merged,self.linear_start_train_step],feed_dict={self.story_batch:np.array(batch_train)[:,0].tolist(),
                                                                  self.query_batch:np.array(np.array(batch_train)[:,1].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                                                  self.answer_batch:np.array(np.array(batch_train)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                                                  self.memory_init_flag_batch:np.array(batch_train)[:,3].tolist()})
                else:
                    summary,_=self.session.run([self.merged,self.train_step],feed_dict={self.story_batch:np.array(batch_train)[:,0].tolist(),
                                                                  self.query_batch:np.array(np.array(batch_train)[:,1].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                                                  self.answer_batch:np.array(np.array(batch_train)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                                                  self.memory_init_flag_batch:np.array(batch_train)[:,3].tolist()})
                self.writer.add_summary(summary,ep*num_train+idx)
            if (ep+1)%10==0:
                print("epoch:",ep+1)
                save_file=open("./memnet_out/Memorynet_our_data_hop_batch_linear_start_"+str(self.memory_dim)+"_"+str(ep+1)+".txt","w")
                total_cross_ent=0
                total=0
                correct=0
                for idx in range(int(num_train/self.batch_size)):
                    if num_train-idx*self.batch_size<self.batch_size:
                        break
                    batch_train=train_data[idx*self.batch_size:(idx+1)*self.batch_size]
                    temp_cross_ent,predict_list,attention_list=self.session.run([self.cross_entropy_sum,self.predict_list,self.attention_list],feed_dict={self.story_batch:np.array(batch_train)[:,0].tolist(),
                                              self.query_batch:np.array(np.array(batch_train)[:,1].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                              self.answer_batch:np.array(np.array(batch_train)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                              self.memory_init_flag_batch:np.array(batch_train)[:,3].tolist()})
                    total_cross_ent+=temp_cross_ent
                    #print(predict_list)
                    answer_list=np.array(np.array(batch_train)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist()
                    for batch_idx in range(self.batch_size):
                        total+=1
                        if np.argmax(predict_list[batch_idx])==np.argmax(answer_list[batch_idx]):
                            correct+=1
                        for i in range(self.story_max):
                            if max(batch_train[batch_idx][0][i])==0:
                                break
                            save_file.write(str(train_dict[batch_train[batch_idx][-1]][0][i])+"\t")
                            for j in range(self.hop):
                                save_file.write(str(attention_list[batch_idx][j][i])+"\t")
                            save_file.write("\n")
                        save_file.write(str(train_dict[batch_train[batch_idx][-1]][1])+"\n")
                        save_file.write("Our result: "+str(self.vocab[np.argmax(predict_list[batch_idx])])+"\tAnswer: "+str(train_dict[batch_train[batch_idx][-1]][2])+"\n\n")
                        #print(batch_train[batch_idx][1])
                        #print(self.from_bag_of_word_to_sentence(batch_train[batch_idx][1]))
                        #print(self.from_bag_of_word_to_sentence(batch_train[batch_idx][2]))
                        #print(attention_list[0])
                        #print(attention_list[1])
                        #print("*******************")
                save_file.close()
                if total_cross_ent>=before_loss:
                    linear_start_flag=False
                else:
                    before_loss=total_cross_ent
                print("total loss:",total_cross_ent)
                print(total)
                print(correct)
                print(correct/total)
                print("#############################")
                total=0
                correct=0
                for idx in range(int(num_valid/self.batch_size)):
                    if num_valid-idx*self.batch_size<self.batch_size:
                        break
                    batch_valid=valid_data[idx*self.batch_size:(idx+1)*self.batch_size]
                    predict_list=self.session.run(self.predict_list,feed_dict={self.story_batch:np.array(batch_valid)[:,0].tolist(),
                                              self.query_batch:np.array(np.array(batch_valid)[:,1].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                              self.answer_batch:np.array(np.array(batch_valid)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist(),
                                              self.memory_init_flag_batch:np.array(batch_valid)[:,3].tolist()})
                    #print(predict_list)
                    answer_list=np.array(np.array(batch_valid)[:,2].tolist()).reshape(-1,1,len(self.vocab)).tolist()
                    for idx in range(self.batch_size):
                        total+=1
                        if np.argmax(predict_list[idx])==np.argmax(answer_list[idx]):
                            correct+=1
                print(total)
                print(correct)
                print(correct/total)
                print("#############################")
                print("#############################")
    
    def from_bag_of_word_to_sentence(self,bag_of_word):
        return np.array(self.vocab)[np.where(bag_of_word)].tolist()
    
    def fit(self,train_data,epoch=2000):
        train_data,valid_data=self.preprocessing(train_data)
        print("Learning ...")
        for ep in range(epoch):
            for story,query,answer,memory_init_flag in train_data:
                #print(story)
                summary,_=self.session.run([self.merged,self.train_step],feed_dict={self.story:story,self.query:[query],self.answer:[answer],self.memory_init_flag:memory_init_flag})
                self.writer.add_summary(summary,ep)
            if (ep+1)%10==0:
                total=0
                correct=0
                for story,query,answer,memory_init_flag in train_data:
                    total+=1
                    if self.vocab[np.argmax(self.session.run([self.predict],feed_dict={self.story:story,self.query:[query],self.answer:[answer],
                                                                                       self.memory_init_flag:memory_init_flag}))]==self.vocab[np.argmax(answer)]:
                        correct+=1
                print("train")
                print(total)
                print(correct)
                print(correct/total)
                total=0
                correct=0
                for story,query,answer,memory_init_flag in valid_data:
                    total+=1
                    if self.vocab[np.argmax(self.session.run([self.predict],feed_dict={self.story:story,self.query:[query],self.answer:[answer],
                                                                                       self.memory_init_flag:memory_init_flag}))]==self.vocab[np.argmax(answer)]:
                        correct+=1
                print("valid")
                print(total)
                print(correct)
                print(correct/total)
        for story,query,answer,memory_init_flag in train_data:
            print(self.vocab[np.argmax(self.session.run([self.predict],feed_dict={self.story:story,self.query:[query],self.answer:[answer],self.memory_init_flag:memory_init_flag}))])
            print(self.vocab[np.argmax(answer)])
            print("#############################")
        print("#############################")
        for story,query,answer,memory_init_flag in valid_data:
            print(self.vocab[np.argmax(self.session.run([self.predict],feed_dict={self.story:story,self.query:[query],self.answer:[answer],self.memory_init_flag:memory_init_flag}))])
            print(self.vocab[np.argmax(answer)])
            print("#############################")
        
        
        #print(self.session.run([self.embedding_story,self.emdedding_query],feed_dict={self.story:story,self.query:query,self.answer:answer,self.memory_init_flag:memory_init_flag}))
    
    def preprocessing(self,train_data,validation_cut=0.8):
        preprocessed_data=[]
        for story,query,answer,memory_init_flag,idx in train_data:
            story_list=[]
            for sentence in story:
                sentence_vector=[0]*len(self.vocab)
                for word in sentence:
                    sentence_vector[self.vocab.index(word)]=1
                story_list.append(sentence_vector)
            for _ in range(self.story_max-len(story_list)):
                story_list.append([0]*len(self.vocab))
            query_vector=[0]*len(self.vocab)
            for word in query:
                query_vector[self.vocab.index(word)]=1
            answer_vector=[0]*len(self.vocab)
            for word in answer:
                answer_vector[self.vocab.index(word)]=1
            preprocessed_data.append([story_list,query_vector,answer_vector,memory_init_flag,idx])
        #print(preprocessed_data)
        return preprocessed_data[:int(len(preprocessed_data)*validation_cut)],preprocessed_data[int(len(preprocessed_data)*validation_cut):]
    
    
    
    def init_memory(self):
        return tf.constant([[0.0]*self.memory_dim],dtype=tf.float32)
        #self.memory=tf.Variable(initial_value=[[0.0]*self.memory_dim]*self.memory_max,trainable=False)
        #self.memory=[[0.0]*self.memory_dim]*self.memory_max
        #self.memory_pointer=tf.constant(0)

In [ ]:
import numpy as np
a=np.empty([0,3])
b=np.array([1,2,3])
c=np.array([1,2,3,4])
d=np.array([1,2,3,4,5])
print(np.append(a,np.array([[b,c,d]]),0))
#print(np.append(np.append(a,b,0),b,0))
print(a)

In [ ]:
babi_example=[[["Mary moved to the bathroom.","John went to the hallway."],"Where is Mary?", "bathroom",0],
              [["Daniel went back to the hallway.","Sandra moved to the garden."],"Where is Daniel?","hallway",1],
              [["John moved to the office","Sandra journeyed to the bathroom."],"Where is Daniel?","hallway",1],
              [["Mary moved to the hallway.","Daniel travelled to the office."],"Where is Daniel?","office",1],
              [["John went back to the garden.","John moved to the bedroom."],"Where is Sandra?","bathroom",1],
              [["Sandra travelled to the office.","Sandra went to the bathroom."],"Where is Sandra?", "bathroom",0],
              [["Mary went to the bedroom.","Daniel moved to the hallway."],"Where is Sandra?", "bathroom",1],
              [["John went to the garden.","John travelled to the office."],"Where is Sandra?",  "bathroom",1],    
              [["Daniel journeyed to the bedroom.","Daniel travelled to the hallway."], "Where is John?", "office",1],
              [["John went to the bedroom.","John travelled to the office."],"Where is Daniel?", "hallway",1]]
preprocessed=[]
vocab=[]
for story,query,answer,memory_init_flag in babi_example:
    sentence_list=[]
    for sentence in story:
        sentence_list.append(nltk.word_tokenize(sentence))
    for sentence in sentence_list:
        for word in sentence:
            if word not in vocab:
                vocab.append(word)
    query_list=nltk.word_tokenize(query)
    for word in query_list:
        if word not in vocab:
            vocab.append(word)
    answer_list=nltk.word_tokenize(answer)
    for word in answer_list:
        if word not in vocab:
            vocab.append(word)
    preprocessed.append([sentence_list,query_list,answer_list,memory_init_flag])
print(preprocessed[0])
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    a=MN(vocab,10,30,2,sess)
    a.fit(preprocessed)

In [ ]:
0.41786873+0.61164886


In [ ]:
len([[[[0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 0], [[[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 0]])

In [ ]:
# Creates a graph.
d=[[[tf.constant(0),tf.constant(0)],[tf.constant(0),tf.constant(0)]]]*2
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.mul(a, b)
  d[0]=c
  e = tf.mul(d,tf.constant(3.0))
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(e))

In [ ]:
# Creates a graph.
d=[[tf.constant(0.0)]*3]*5
with tf.device('/gpu:0'):
  a = tf.constant([[[1.0, 2.0, 3.0,4.0],[5.0,6.0,7.0,8.0],[8.0,9.0,10.0,11.0]]], name='a')
  b = tf.constant([[3.0, 4.0, 5.0]], name='b')
  #d[0]=tf.cast(a,tf.float32)
  f=tf.matmul(b,a)
  #e = a*[tf.constant(3.0),tf.constant(4.0),tf.constant(5.0)]
  e = tf.matmul(([[tf.constant(3.0),tf.constant(4.0),tf.constant(5.0)]]),a)
  #g=tf.reduce_sum(a*b,1)
  h=b+b
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(e))
print(sess.run(h))

In [ ]:
x = tf.constant(5.0, shape=[5, 6])
w = tf.constant([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])
xw = tf.mul(x, w)
max_in_rows = tf.reduce_max(xw, 1)

sess = tf.Session()
print(sess.run(xw))

In [ ]:
np.argmax([1,2,3,4])

In [ ]:
!nvidia-smi